In [1]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

from models.cnn import mini_XCEPTION
from utils.datasets import DataManager
from utils.preprocessor import preprocess_input
###############################################
import keras
from keras.models import Sequential
from keras.models import Model
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import  BatchNormalization
from keras.layers.convolutional import *
from keras import regularizers ### for SVM
import numpy as np
import tensorflow as tf
# import theano
import  matplotlib.pyplot as plt
import matplotlib
import itertools
from numpy import*
from sklearn.utils import shuffle
%matplotlib inline
import random as rn
import os
os.environ['PYTHONASHSEED']= '0'
from keras.models import load_model

tf.reset_default_graph()

Using TensorFlow backend.


In [2]:
#setting  the  seed for nummy_gennerated random numbers
np.random.seed(7)
#seting the seed for python random numbers
rn.seed(124)
#seting the seed for tensorflow random numbers
tf.set_random_seed(57)

In [3]:
train_path = '.\MAKEUP\Train' 
valid_path = '.\MAKEUP\Valid' 
#test_path = '.\CASIA1\Test' 

In [4]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = (64,64),color_mode='grayscale', classes = ['MakeUp','Normal'], batch_size =8)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size = (64,64),color_mode='grayscale', classes = ['MakeUp','Normal'], batch_size = 8)

Found 604 images belonging to 2 classes.
Found 154 images belonging to 2 classes.


In [5]:
input_shape = (64, 64, 1)
num_classes = 7
model = mini_XCEPTION(input_shape, num_classes)

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 62, 62, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 62, 62, 8)    32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 62, 62, 8)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [7]:
model.load_weights('fer2013_mini_XCEPTION.hdf5') 

In [8]:
model.layers.pop()
last = model.layers[-1].output
x = Dense(2, activation = 'linear', kernel_regularizer=regularizers.l2(0.001))(last)
finetuned_model = Model(model.input, x)

In [9]:
n = len(finetuned_model.layers)
for idx, layer in enumerate(finetuned_model.layers):
    if idx < (n-2): 
           layer.trainable = False   
    else:
           layer.trainable = True   
          

In [10]:
opt = Adam(lr=0.0001, decay=10e-6)
finetuned_model.compile(loss='hinge', optimizer=opt, metrics=['accuracy'])

In [11]:
from keras.callbacks import ModelCheckpoint
# Save check point
filepath = "weights.resnet50.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose =1, save_best_only = True, mode ='max')
callbacks_list= [checkpoint]
 # Fit the model
# model.fit(X,Y, validation_split=0.33, nb_epoch =150, batch_size =10, callbacks = callbacks_list, verbose =0)

history = finetuned_model.fit_generator(train_batches, steps_per_epoch = 120, validation_data = valid_batches, validation_steps=26, epochs =200, shuffle=True, callbacks = callbacks_list, verbose =2)

Epoch 1/200
 - 2s - loss: 1.5464 - acc: 0.5000 - val_loss: 155.5181 - val_acc: 0.4604

Epoch 00001: val_acc improved from -inf to 0.46040, saving model to weights.resnet50.best.hdf5
Epoch 2/200
 - 1s - loss: 1.5370 - acc: 0.4844 - val_loss: 138.7606 - val_acc: 0.5198

Epoch 00002: val_acc improved from 0.46040 to 0.51980, saving model to weights.resnet50.best.hdf5
Epoch 3/200
 - 1s - loss: 1.4903 - acc: 0.4969 - val_loss: 145.8855 - val_acc: 0.4950

Epoch 00003: val_acc did not improve
Epoch 4/200
 - 1s - loss: 1.4626 - acc: 0.4906 - val_loss: 145.3960 - val_acc: 0.5000

Epoch 00004: val_acc did not improve
Epoch 5/200
 - 1s - loss: 1.4332 - acc: 0.4937 - val_loss: 147.3446 - val_acc: 0.4950

Epoch 00005: val_acc did not improve
Epoch 6/200
 - 1s - loss: 1.3900 - acc: 0.4948 - val_loss: 147.8235 - val_acc: 0.4950

Epoch 00006: val_acc did not improve
Epoch 7/200
 - 1s - loss: 1.3640 - acc: 0.4885 - val_loss: 144.0656 - val_acc: 0.5099

Epoch 00007: val_acc did not improve
Epoch 8/200
 